In [4]:
# Header cell
from __future__ import division
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

# Specific things needed
import time
import math
import sys

# Add parent directory to path
sys.path.append('../code/')
sys.path.append('../sim/')

# Import deft modules
import deft_1d
import simulate_data_1d
import utils
from utils import DeftError

In [5]:
class TestCase:
    ''' Contains information needed for a test run of deft_1d. '''
    
    def __init__(self, G, alpha, bbox, N, data_type, should_succeed=True, 
                 num_samples=0, deft_seed=None, data_seed=None):
        
        # Set internal variables
        self.G = G
        self.alpha = alpha
        self.bbox = bbox
        self.N = N
        self.data_type = data_type
        self.should_succeed = should_succeed
        self.num_samples = num_samples
        self.data_seed = data_seed
        self.deft_seed = deft_seed
        self.outcome_good = False
        
    def __repr__(self):
        return 'G=%d, alpha=%d, bbox=%s, N=%d, data_type=%s, should_succeed=%s'%\
            (self.G, self.alpha, self.bbox, self.N, self.data_type, self.should_succeed)
        
    def run(self):
        
        # Set other variables
        periodic = False
        Laplace = False
        num_steps_per_sample = self.G
        num_thermalization_steps = 10*self.G
        fix_t_at_t_star = False
        print_t = False
        tollerance = 1E-3
        resolution = 1E-1
        
        # Generate data
        self.data, self.defaults = simulate_data_1d.run(self.data_type, self.N, self.data_seed)
        
        # Run deft_1d
        print self
        
        try:
            self.results = deft_1d.run(self.data, G=self.G, alpha=self.alpha, bbox=self.bbox, \
                                       periodic=periodic, Laplace=Laplace, num_samples=self.num_samples, \
                                       num_steps_per_sample=num_steps_per_sample, \
                                       num_thermalization_steps=num_thermalization_steps, \
                                       fix_t_at_t_star=fix_t_at_t_star, print_t=print_t, \
                                       tollerance=tollerance, resolution=resolution, seed=self.deft_seed)
            print 'Succeeded!'
            self.outcome_good = self.should_succeed

        except DeftError:
            print 'Failed in controlled way.'
            self.outcome_good = not self.should_succeed
            
        #except:
        #    print 'Problem! Test failed in an uncontrolled way!'
        #    self.outcome_good = False
            
        if self.outcome_good:
            print 'Outcome: good.'
        else:
            print 'Outcome: BAD!'
        print ''

In [6]:
test_cases = [
    TestCase(G=100,alpha=3, bbox=[-6,6,1], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=False),
    TestCase(G=100,alpha=3, bbox=[6,-6], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=False),
    TestCase(G=100,alpha=3, bbox=[-6,6], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=True),
    TestCase(G=100,alpha=-3, bbox=[-6,6], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=False),
    TestCase(G=-100,alpha=3, bbox=[-6,6], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=False),
    TestCase(G=0,alpha=3, bbox=[-6,6], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=False),
    TestCase(G=100.5,alpha=3, bbox=[-6,6], N=100, data_type='wide', deft_seed=0, data_seed=0, should_succeed=False)
#    TestCase(G=100,alpha=3, bbox=[-6,6], N=100, data_type='wide', deft_seed=0, data_seed=1)
#    TestCase(G=100,alpha=1, bbox=[-6,6], N=100, data_type='wide'),
#    TestCase(G=100,alpha=3, bbox=[-20,20], N=100, data_type='wide'),
#    TestCase(G=100,alpha=3, bbox=[-100,100], N=100, data_type='wide'),
#    TestCase(G=100,alpha=3, bbox=[6,-6], N=100, data_type='wide', should_succeed=False),
#    TestCase(G=100,alpha=3, bbox=[-6,6], N=0, data_type='wide', should_succeed=False)
]
#test_cases = [TestCase(G=100,alpha=3, bbox=[-6,6], N=100, data_type='wide', deft_seed=n, data_seed=0)\
#             for n in range(10)]

for count, test_case in enumerate(test_cases):
    print 'Test case #', count
    test_case.run()    

Test case # 0
G=100, alpha=3, bbox=[-6, 6, 1], N=100, data_type=wide, should_succeed=False
Failed in controlled way.
Outcome: good.

Test case # 1
G=100, alpha=3, bbox=[6, -6], N=100, data_type=wide, should_succeed=False
Failed in controlled way.
Outcome: good.

Test case # 2
G=100, alpha=3, bbox=[-6, 6], N=100, data_type=wide, should_succeed=True
Succeeded!
Outcome: good.

Test case # 3
G=100, alpha=-3, bbox=[-6, 6], N=100, data_type=wide, should_succeed=False
Failed in controlled way.
Outcome: good.

Test case # 4
G=-100, alpha=3, bbox=[-6, 6], N=100, data_type=wide, should_succeed=False


AssertionError: 

In [ ]:
plt.figure()
for n, test_case in enumerate(test_cases):
    print 'Test case %d: t_star = %f'%(n,test_case.results.t_star)
    Q_star = test_case.results.Q_star
    xs = test_case.results.bin_centers
    plt.plot(xs,Q_star)

In [ ]:
result = test_case.results


In [ ]:
# Plot Prob(t) vs t
if (test_case.num_samples > 0):
    plt.figure(figsize=[8,6])
    x = results.prob_t_vs_t[0,:]
    y = results.prob_t_vs_t[1,:]
    plt.scatter(x, y)
    plt.plot(x, y, color='red', linewidth=1)
    #plt.semilogy(x, y, color='red', linewidth=1)
    plt.xlim(x.min()+10, x.max()-10)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.xlabel('t (-inf, +inf)', size=20)
    plt.ylabel('Prob ~ E(t)', size=20)

In [ ]:
# Plot log_E(t) vs t
if (test_case.num_samples > 0):
    plt.figure(figsize=[8,6])
    x = results.prob_t_vs_t[0,:]
    y = results.prob_t_vs_t[2,:]
    plt.scatter(x, y)
    plt.plot(x, y, color='red', linewidth=1)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.xlabel('t (-inf, +inf)', size=20)
    plt.ylabel('log_E(t)', size=20)

In [ ]:
# Compute true density
xs = results.bin_centers
Q_true = np.zeros(test_case.G)
for i, x in enumerate(xs):
    Q_true[i] = eval(test_case.defaults['pdf_py'])
Q_true /= results.h*sum(Q_true)

In [ ]:
plt.figure(figsize=[12,9])
xs = results.bin_centers

# plot histogram density
left_bin_edges = results.bin_edges[:-1]
plt.bar(xs, results.R, width=results.h, color='gray', linewidth=0, zorder=0, alpha=0.5) # Here I used xs!

# Plot the MCMC samples from S
if (test_case.num_samples > 0):
    plt.plot(xs, results.Q_samples, color='blue', linewidth=5, zorder=1, alpha=0.1)

# Plot DEFT density estimate
plt.plot(xs, results.Q_star, color='red', linewidth=3, zorder=3, alpha=1.0)

# Plot the true density
plt.plot(xs, Q_true, color='black', linewidth=2, zorder=2, alpha=1.0)

# Tidy up the plot
plt.ylim(0, 1.1*max(max(results.R),max(results.Q_star)))
plt.xlim(test_case.bbox)
plt.yticks(size=20)
plt.xticks(size=20)
plt.ylabel('Probability density', size=25)
plt.xlabel('Data', size=25)
t = results.deft_1d_compute_time
plt.title('%s, t=%1.2f sec %s'%(test_case.data_type, t, '(Black=Q_true, Red=Q_star, Blue=Q_samples)'), \
          fontsize=20)